https://neurohive.io/ru/osnovy-data-science/razbor-resheniya-zadachi-titanik-na-kaggle-dlya-nachinajushhih/  
https://github.com/sumitmukhija/Titanic/blob/master/Tip%20of%20the%20Iceberg.ipynb  
https://www.kaggle.com/konstantinmasich/titanic-0-82-0-83/notebook  
https://github.com/ramansah/kaggle-titanic/blob/master/Analysis.ipynb  
https://www.kaggle.com/startupsci/titanic-data-science-solutions  
https://www.kaggle.com/makarevich/missing-values-that-you-can-t-miss-in-titanic  
https://www.kaggle.com/vradore/titanic-survival-analysis-with-missing-values  
https://www.kaggle.com/c/titanic/discussion/62321  

In [ ]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_len = len(train_data)
df = pd.concat([train_data, test_data])
df['Last_Name'] = df['Name'].str.split(",").str.get(0)

for name, group in df.groupby(['Pclass','Last_Name']):
    if len(group) > 9: 
        print(group[['Last_Name','Cabin','Ticket']])
        print(len(group))

In [ ]:
one_hot_encoded = ['Pclass','Sex','Age','SibSp','Parch','Cabin','Embarked','Fare','Alone']
X_train = pd.get_dummies(X_train, columns=one_hot_encoded)
X_test = pd.get_dummies(X_test, columns=one_hot_encoded)
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

In [10]:
import pandas as pd
df1 = pd.read_csv('titanic.csv')
df2 = pd.read_csv('test.csv')
df1 = df1.drop(['Boat','Body','Home.dest'], axis=1)
df = pd.concat([df1, df2])
df = df[df.duplicated(['Name'], keep=False)]
df = df.sort_values(by='Name')
df = df[df['Survived'].notnull()]
df.to_csv('my_test.csv', index=False)

In [ ]:
df1 = pd.read_csv('titanic.csv')
df2 = pd.read_csv('train.csv')
df3 = pd.read_csv('test.csv')
df4 = pd.concat([df1,df2]).drop_duplicates(subset='Name', keep="last")
df1.shape, df2.shape, df3.shape, df4.shape

In [ ]:
from sklearn.model_selection import GridSearchCV

m = LogisticRegression()

grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1','l2'],
    'solver': ['liblinear']
}

cv = GridSearchCV(m, grid, scoring='accuracy', cv=3)
cv.fit(X_train, y_train)
cv.best_estimator_, cv.best_score_

In [ ]:
train_data['Name'].str.contains('Don').sum()

In [ ]:
(train_data['Fare'] / (train_data['SibSp'] + train_data['Parch'] + 1) < 1).sum()

In [ ]:
train_data['Pclass'].isnull().sum()

In [ ]:
train_data.groupby('Sex').apply(lambda x: x['Survived'].sum() / len(x))

In [ ]:
train_data.Survived[train_data['Sex'] == 'male'].value_counts().plot(kind='pie')

In [ ]:
train_data.Survived[train_data['Sex'] == 'female'].value_counts().plot(kind='pie')

In [ ]:
train_data.loc[train_data['Sex'] == 'male', 'Survived'].mean()
train_data.loc[train_data['Sex'] == 'female', 'Survived'].mean()

In [ ]:
train_data['Age_c'] = pd.cut(train_data['Age'], 4)
# train_data['Age_c'] = pd.cut(train_data['Age'], bins=[0,7.90,14.45,31.28,120])
s = train_data.groupby('Age_c').apply(lambda x: x['Survived'].sum() / len(x))
plt.bar(s.index.astype(str), s.values)
plt.xticks(rotation=45);

In [ ]:
women = train_data[train_data['Sex']=='female']
men = train_data[train_data['Sex']=='male']
# women[women['Survived']==0].Age.dropna().hist()
women[women['Survived']==1].Age.dropna().hist()

In [ ]:
#women = train_data[train_data['Sex']=='female']
#men = train_data[train_data['Sex']=='male']
#sns.displot(men[men['Survived']==1].Age.dropna(), bins=18)
#sns.displot(men[men['Survived']==0].Age.dropna(), bins=18)
#sns.displot(women[women['Survived']==1].Age.dropna(), bins=18)
#sns.displot(women[women['Survived']==0].Age.dropna(), bins=18)

In [ ]:
s = train_data.groupby('Sex').apply(lambda x: x['Survived'].sum() / len(x))
plt.bar(s.index.astype(str), s.values);

In [ ]:
train_data['Age_c'] = pd.cut(train_data['Age'], 5)
train_data.groupby('Age_c').apply(lambda x: x['Survived'].sum() / len(x))

In [ ]:
s = train_data.groupby('Pclass').apply(lambda x: x['Survived'].sum() / len(x))
plt.bar(s.index.astype(str), s.values);

In [ ]:
train_data['n'] = train_data.groupby('Ticket')['Fare'].transform(lambda x: len(x))
train_data['Fare1'] = train_data.groupby('Ticket')['Fare'].transform(lambda x: x / len(x))
train_data['Fare1'] = pd.cut(train_data['Fare1'], bins=[0,10,20,30,100])

In [ ]:
import matplotlib.pyplot as plt
s = train_data.groupby('Fare1').apply(lambda x: x['Survived'].sum() / len(x))
plt.bar(s.index.astype(str), s.values);
s

In [ ]:
# train_data['Age'].isnull().sum()
# train_data['Title'].isnull().sum()
# X['Age'] = X['Age'].fillna(X['Age'].mean())
train_data['Title'] = train_data['Name'].str.extract('(Mrs|Mr|Miss)').fillna('NA')
train_data['Age'] = train_data.groupby('Title').transform(lambda g: g.fillna(g.mean()))
# train_data['Age'] = train_data.groupby('Title')['Age'].apply(lambda g: g.fillna(g.median()))

In [ ]:
X = train_data
X['Title'] = X['Name'].str.extract('([A-Za-z]+)\.').fillna('NA')
# X['Title'] = X['Title'].map({'Mlle':'Miss','Major':'Mr','Col':'Mr','Sir':'Mr','Don':'Mr','Mme':'Miss','Lady':'Mrs','Jonkheer':'Mr','Capt':'Mr','Countess':'Mrs','Ms':'Miss','Dona':'Mrs'})           

X['Title'].unique()

X['Title'].replace({'Mlle':'Miss','Major':'Mr','Col':'Mr','Sir':'Mr','Don':'Mr','Mme':'Miss','Lady':'Mrs','Jonkheer':'Mr','Capt':'Mr','Countess':'Mrs','Ms':'Miss','Dona':'Mrs'})
X['Age'] = X.groupby(['Pclass','Title'])['Age'].transform(lambda g: g.fillna(g.mean()))
X['Age']

In [ ]:
X['Age'] = pd.cut(X['Age'], bins=[0,7.90,14.45,31.28,120])
X['Fare'] = pd.cut(X['Fare'], bins=[0,10,20,30,100])

pd.qcut(train_data['Fare'], 5, labels=[1,2,3,4,5]) # without one hot
pd.qcut(train_data['Age'], 4, labels=[1,2,3,4]) # without one hot
train_data['Sex'].map({'male':0,'female':1}) # without one hot

from sklearn.preprocessing import LabelEncoder
#label = LabelEncoder()
#train_data['Fare'] = label.fit_transform(train_data['Fare'])
#train_data['Fare']

In [ ]:
import matplotlib.pyplot as plt

train_data = pd.read_csv('train.csv')

train_data['Title'] = train_data['Name'].str.extract('([A-Za-z]+)\.').fillna('NA')
train_data['Title'] = train_data['Title'].replace({'Mlle':'Miss','Major':'Mr','Col':'Mr','Sir':'Mr','Don':'Mr',
       'Mme':'Miss','Lady':'Mrs','Jonkheer':'Mr','Capt':'Mr','Countess':'Mrs','Ms':'Miss','Dona':'Mrs'})

train_data['Age'] = train_data.groupby(['Pclass','Title'])['Age'].transform(lambda g: g.fillna(g.mean()))
train_data['Age'] = pd.cut(train_data['Age'], 10) # bins=[0,7.90,14.45,31.28,120]

s = train_data.groupby('Age').apply(lambda x: x['Survived'].sum() / len(x))
plt.bar(s.index.astype(str), s.values);

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

y_train = train_data['Survived'].copy()
X_train = train_data[['Pclass','Sex','SibSp','Parch','Age','Fare','Embarked','Cabin']].copy()
X_test = test_data[['Pclass','Sex','SibSp','Parch','Age','Fare','Embarked','Cabin']].copy()

X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].mean())
X_train['Fare'] = X_train['Fare'].fillna(X_train['Fare'].mean())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].mean())
X_train['Embarked'] = X_train['Fare'].fillna(X_train['Embarked'].mode()[0])
X_test['Embarked'] = X_test['Fare'].fillna(X_test['Embarked'].mode()[0])
X_train['Cabin'] = X_train['Cabin'].fillna('NA')
X_test['Cabin'] = X_test['Cabin'].fillna('NA')

X_train['Family'] = X_train['Parch'] + X_train['SibSp']
X_train['Is_Alone'] = (X_train['Family'] == 0)

X_train['Age'] = pd.cut(X_train['Age'], 80, labels=range(80))
X_test['Age'] = pd.cut(X_test['Age'], 80, labels=range(80))
X_train['Fare'] = pd.cut(X_train['Fare'], bins=[0,7.90,14.45,31.28,120], labels=range(4))
X_test['Fare'] = pd.cut(X_test['Fare'], bins=[0,7.90,14.45,31.28,120], labels=range(4))

# X_train['Sex'] = X_train['Sex'].map({'male':0,'female':1})
# X_test['Sex'] = X_test['Sex'].map({'male':0,'female':1})
X_train['Sex'] = LabelEncoder().fit_transform(X_train['Sex'])
X_test['Sex'] = LabelEncoder().fit_transform(X_test['Sex'])
X_train['Embarked'] = LabelEncoder().fit_transform(X_train['Embarked'])
X_test['Embarked'] = LabelEncoder().fit_transform(X_test['Embarked'])
X_train['Cabin'] = LabelEncoder().fit_transform(X_train['Cabin'])
X_test['Cabin'] = LabelEncoder().fit_transform(X_test['Cabin'])

# scaler = StandardScaler()
# X_train[['Age']] = scaler.fit_transform(X_train[['Age']])
# X_test[['Age']] = scaler.transform(X_test[['Age']])

# scaler = MinMaxScaler()
# X_train[['Fare']] = scaler.fit_transform(X_train[['Fare']])
# X_test[['Fare']] = scaler.transform(X_test[['Fare']])

X_train = pd.get_dummies(X_train, columns=['Pclass','Sex','SibSp','Parch','Age','Fare','Embarked','Cabin'])
X_test = pd.get_dummies(X_test, columns=['Pclass','Sex','SibSp','Parch', 'Age', 'Fare','Embarked','Cabin'])
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

lr = LogisticRegression(C=1, solver='liblinear')
lr.fit(X_train, y_train)

output = pd.DataFrame()
output['PassengerId'] = test_data.PassengerId
output['Survived'] = lr.predict(X_test)
output.to_csv('my_submission.csv', index=False)

cross_val_score(lr, X_train, y_train, scoring='accuracy', cv=3).mean()
# accuracy_score(y_train, lr.predict(X_train))

output.shape

In [ ]:
X_test.shape

In [ ]:
lr = LogisticRegression()

grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1','l2'],
    'solver': ['liblinear']
}

cv = GridSearchCV(lr, grid, scoring='accuracy', cv=3)
cv.fit(X_train, y_train)
cv.best_estimator_, cv.best_score_

In [ ]:
plt.plot(train_data['Survived'], train_data['Fare'], '.')

In [ ]:
train_data['Fare'].hist()

In [ ]:
train_data.groupby('Fare')['Survived'].sum()

In [ ]:
# X_test = X_test.drop('Parch_9', axis=1)
# train_data[train_data['Age'].isnull()]
# train_data['Cabin'].unique()
# (test_data['Fare'] == 0).sum()
# X_train['Age'].mean()
# pd.cut(train_data['Age'], 5, labels=[1,2,3,4,5])
# X_train.isnull().sum()

In [ ]:
def age_category(age):
    if age < 16: return 1
    if age < 35: return 2
    return 3

# train_data['Age'].apply(age_category)

In [ ]:
'''
def age_category(age):
    if age < 18: return 1
    if age < 35: return 2
    return 3

X_train['Age'] = X_train['Age'].apply(age_category)
X_test['Age'] = X_test['Age'].apply(age_category)
'''

In [ ]:
# X_train[:5]

In [ ]:
'''
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

features = ['Pclass','Sex','SibSp','Parch']
y_train = train_data['Survived'].copy()

X_train = train_data[features].copy()
X_train['Sex'] = X_train['Sex'].map({'male':0,'female':1})
X_train = pd.get_dummies(X_train, columns=features)

X_test = test_data[features].copy()
X_test['Sex'] = X_test['Sex'].map({'male':0,'female':1})
X_test = pd.get_dummies(X_test, columns=features)
X_test = X_test.drop('Parch_9', axis=1)

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
output = pd.DataFrame()
output['PassengerId'] = test_data.PassengerId
output['Survived'] = knn.predict(X_test)
output.to_csv('my_submission.csv', index=False)
'''

In [ ]:
# train_data[:5]

In [ ]:
# train_data.groupby('Age').size().sort_index().plot()
# plt.plot(s.index, s.values)

In [ ]:
# train_data.groupby('Sex').size()

In [ ]:
'''
accuracy_score(y_train, knn.predict(X_train))
accuracy_score(y_train, lr.predict(X_train))
sns.pairplot(train_data, hue='Survived')
plt.scatter(x=train_data['SibSp'], y=train_data['Pclass'])
train_data['Sex'].hist()
cross_val_score(lr, X_train, y_train, scoring='accuracy', cv=3)
confusion_matrix(y_train, lr.predict(X_train))
cat my_submission.csv
lr.coef_
X_train.isnull().sum()
X_train.info()
X_train = X_train.replace(['male','female'], [0,1])
X_train['Sex'] = X_train['Sex'].replace(['male','female'], [0,1])
X_train['Sex'] = X_train['Sex'].replace({'male':0,'female':1})
X_train['Sex'] = X_train['Sex'].map({'male':0,'female':1})
X_train['Sex'] = 1 #X_train['Sex'].apply(lambda x: {'male':0,'female':1}[x])
X_train['Sex'].isnull().sum()
train_data[:5]
'''

In [ ]:
'''
from sklearn.ensemble import RandomForestClassifier
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
'''

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

y_train = train_data['Survived'].copy()
X_train = train_data[['Pclass','Sex','SibSp','Parch','Age','Fare','Embarked','Cabin']].copy()
X_test = test_data[['Pclass','Sex','SibSp','Parch','Age','Fare','Embarked','Cabin']].copy()

X_train['Age'] = X_train['Age'].fillna(X_train['Age'].mean())
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].mean())
X_train['Fare'] = X_train['Fare'].fillna(X_train['Fare'].mean())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].mean())
X_train['Embarked'] = X_train['Embarked'].fillna(X_train['Embarked'].mode()[0])
X_test['Embarked'] = X_test['Embarked'].fillna(X_test['Embarked'].mode()[0])
X_train['Cabin'] = X_train['Cabin'].fillna('NA')
X_test['Cabin'] = X_test['Cabin'].fillna('NA')

X_train['Family'] = X_train['Parch'] + X_train['SibSp']
X_train['Is_Alone'] = (X_train['Family'] == 0)

X_train['Age'] = pd.cut(X_train['Age'], 80, labels=range(80))
X_test['Age'] = pd.cut(X_test['Age'], 80, labels=range(80))
X_train['Fare'] = pd.cut(X_train['Fare'], bins=[0,7.90,14.45,31.28,120], labels=range(4))
X_test['Fare'] = pd.cut(X_test['Fare'], bins=[0,7.90,14.45,31.28,120], labels=range(4))

# X_train['Sex'] = X_train['Sex'].map({'male':0,'female':1})
# X_test['Sex'] = X_test['Sex'].map({'male':0,'female':1})
X_train['Sex'] = LabelEncoder().fit_transform(X_train['Sex'])
X_test['Sex'] = LabelEncoder().fit_transform(X_test['Sex'])
X_train['Embarked'] = LabelEncoder().fit_transform(X_train['Embarked'])
X_test['Embarked'] = LabelEncoder().fit_transform(X_test['Embarked'])
X_train['Cabin'] = LabelEncoder().fit_transform(X_train['Cabin'])
X_test['Cabin'] = LabelEncoder().fit_transform(X_test['Cabin'])

# scaler = StandardScaler()
# X_train[['Age']] = scaler.fit_transform(X_train[['Age']])
# X_test[['Age']] = scaler.transform(X_test[['Age']])

# scaler = MinMaxScaler()
# X_train[['Fare']] = scaler.fit_transform(X_train[['Fare']])
# X_test[['Fare']] = scaler.transform(X_test[['Fare']])

X_train = pd.get_dummies(X_train, columns=['Pclass','Sex','SibSp','Parch','Age','Fare','Embarked','Cabin'])
X_test = pd.get_dummies(X_test, columns=['Pclass','Sex','SibSp','Parch', 'Age', 'Fare','Embarked','Cabin'])
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

lr = LogisticRegression(C=1, solver='liblinear')
lr.fit(X_train, y_train)

output = pd.DataFrame()
output['PassengerId'] = test_data.PassengerId
output['Survived'] = lr.predict(X_test)
output.to_csv('my_submission.csv', index=False)

cross_val_score(lr, X_train, y_train, scoring='accuracy', cv=3).mean()
# accuracy_score(y_train, lr.predict(X_train))

output.shape